# Imatinib

### Libraries

In [1]:
import sys

sys.path.append("..")

In [2]:
import open3d as o3d
import numpy as np
import seaborn as sns
import pandas as pd

import os
from io import StringIO


import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem

import ipywidgets as widgets
from IPython.display import HTML

from open3d.web_visualizer import draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-07-20 13:09:41,258 - __init__ - Enabling RDKit 2021.03.3 jupyter extensions


[Open3D INFO] Resetting default logger to print to terminal.


In [3]:
from score_pcd import fit_and_score

## Create Point Clouds

Create PCD files for imatinib using Singularity container:

In [4]:
%%bash

singularity run --nv --app python ../development/densitymatch.sif \
    ../molgrid_to_pcd.py ../files/imatinib.sdf -m ../files/ligmap -r 0.5 -o ligands/imatinib.pcd
singularity run --nv --app python ../development/densitymatch.sif \
    ../molgrid_to_pcd.py ../files/imatinib_moved.sdf -m ../files/ligmap -r 0.5 -o ligands/imatinib_moved.pcd
singularity run --nv --app python ../development/densitymatch.sif \
    ../molgrid_to_pcd.py ../files/imatinib_part.sdf -m ../files/ligmap -r 0.5 -o ligands/imatinib_part.pcd

Using a Singularity container to run `molgrid_to_pcd.py` is needed because of a [`openbabel`/`molgrid` incomparibility from PyPI](https://github.com/gnina/libmolgrid/issues/62). `molgrid` and `openbabel` in the Singularity container are both installed form source.

## Imatinib Alignment

Load SDF and PCD files for imatibib (original coordinates and moved coordinates):

In [5]:
files = ["imatinib.pcd", "imatinib_moved.pcd"]

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(os.path.join("ligands", f))
    pcds.append(pcd)

    s = Chem.SDMolSupplier(os.path.join("../files",f.replace(".pcd", ".sdf")))
    mol = next(s)
    mols.append(mol)

In [6]:
print(pcds, mols)

[PointCloud with 1284 points., PointCloud with 1239 points.] [<rdkit.Chem.rdchem.Mol object at 0x7fab1279e8a0>, <rdkit.Chem.rdchem.Mol object at 0x7fab127a9760>]


We can visualize the point clouds using Open3D (does not work on VSCode Jupyter extension for some reason...):

In [7]:
# This fails...
#draw(pcds[0])

In [8]:
# This fails...
#draw(pcds[1])

We can visualize the molecules using `py3Dmol`:

In [9]:
p = py3Dmol.view()

for i, mol in enumerate(mols):
    mb = Chem.MolToMolBlock(mol, confId=0)
    p.addModel(mb,'sdf')

# Select model in order of additon with the following selection: {"model": index}
p.setStyle({"model": 0}, {'stick':{'colorscheme':'lightgreyCarbon'}})
p.setStyle({"model": 1}, {'stick':{"colorscheme": "greyCarbon"}})


p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We can now fit the imatinib (as point cloud) into imatinib and score the alignmenment:

In [10]:
gfit, cfit, tran = fit_and_score(pcds, voxel_size=0.5, threshold=0.5)

In [11]:
print("GFIT:", gfit.fitness)
print("CFIT:", cfit.fitness)
print("transformation:\n", tran)

GFIT: 0.7562305295950156
CFIT: 0.9587227414330218
transformation:
 [[ -0.22349726  -0.96928013  -0.10268889   4.2042644 ]
 [ -0.72704401   0.09561122   0.6799011    0.47138654]
 [ -0.64919641   0.22661538  -0.72607816 -10.65679689]
 [  0.           0.           0.           1.        ]]


The alignment also returns a $4\times 4$ representing the affine transformation (rotation and translation) needed to superimpose the two point clouds (and molecules). The affine transformation is defined by the following $4\times 4$ matrix:
$$
    A = \begin{pmatrix}
        r_{00} & r_{01} & r_{02} & t_0 \\
        r_{10} & r_{11} & r_{12} & t_1 \\
        r_{20} & r_{21} & r_{22} & t_2 \\
        0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$
For an affine tranfromation $A$ represented in this way, the transformed points $\mathbf{x}'$ are given by:
$$
    \begin{pmatrix}
        x' \\ y' \\ z' \\ 1
    \end{pmatrix} = 
    A \begin{pmatrix}
        x \\ y \\ z \\ 1
    \end{pmatrix}
$$

We can now transform the corrdinates of the RDKit molecules in order to check the superimposition:

In [12]:
# Molecule to transform
molid = 0

In [13]:

# Select coordinates to transform
coords = mols[molid].GetConformer(0).GetPositions()

# Augment coordinates with ones
coords_aug = np.ones((coords.shape[0], 4))
coords_aug[:,:3] = coords

# Transform coordinates (using broadcasting)
coords_new = np.matmul(tran, coords_aug.T)[:3,:].T

In [14]:
# Add new coordinates as additional conformer

n_atoms = mols[molid].GetNumAtoms()
conf = Chem.Conformer(n_atoms)
for i in range(n_atoms):
    conf.SetAtomPosition(i, coords_new[i,:])

_ = mols[molid].AddConformer(conf, assignId=True)

We can now visually check the alignment:

In [15]:
p = py3Dmol.view()

for i, mol in enumerate(mols[:2]):
    confid = 0
    if i == molid:
        confid = 1
    
    p.addModel(Chem.MolToMolBlock(mol, confId=confid),'sdf')

p.setStyle({"model": 0}, {'stick':{'colorscheme':'redCarbon'}})
p.setStyle({"model": 1}, {'stick':{'colorscheme':'lightgreyCarbon'}})


p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
#print(Chem.MolToMolBlock(mols[0],confId=0))
#print(Chem.MolToMolBlock(mols[0],confId=1))
#print(Chem.MolToMolBlock(mols[1],confId=0))

## Imatininib Alignment with Subset

We can now try to align a subset (a small fragment) of imatinib into the whole molecule:

In [17]:
files = ["imatinib.pcd", "imatinib_part.pcd"]

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(os.path.join("ligands", f))
    pcds.append(pcd)

    s = Chem.SDMolSupplier(os.path.join("../files",f.replace(".pcd", ".sdf")))
    mol = next(s)
    mols.append(mol)

In [18]:
print(pcds, mols)

[PointCloud with 1284 points., PointCloud with 477 points.] [<rdkit.Chem.rdchem.Mol object at 0x7fab127b2620>, <rdkit.Chem.rdchem.Mol object at 0x7fab127b2710>]


In [19]:
# This fails in VSCode
#draw(pcds[0])
#draw(pcds[1])

We can see that initially the fragment is completely misaligned:

In [20]:
p = py3Dmol.view()

for i, mol in enumerate(mols):
    mb = Chem.MolToMolBlock(mol, confId=0)
    p.addModel(mb,'sdf')
    
p.setStyle({"model": 0}, {'stick':{'colorscheme':'lightgreyCarbon'}})
p.setStyle({"model": 1}, {'stick':{'colorscheme':'redCarbon'}})

p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [21]:
gfit, cfit, tran = fit_and_score(pcds, voxel_size=0.5, threshold=0.5)

In [22]:
print("GFIT:", gfit.fitness)
print("CFIT:", cfit.fitness)
print("transformation:\n", tran)

GFIT: 0.22118380062305296
CFIT: 0.37149532710280375
transformation:
 [[-8.89661697e-01  4.16736286e-01 -1.86635828e-01 -4.38664352e+00]
 [ 1.72422482e-01 -7.18748526e-02 -9.82397319e-01 -1.19969966e+00]
 [-4.22815033e-01 -9.06181479e-01 -7.91040162e-03  1.03934250e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [23]:
# Molecule to transform
molid = 0

# Get coordinates to transform
coords = mols[molid].GetConformer(0).GetPositions()

# Augment coordinates with ones
coords_aug = np.ones((coords.shape[0], 4))
coords_aug[:,:3] = coords

coords_new = np.matmul(tran, coords_aug.T)[:3,:].T

# Add new coordinates as conformer
n_atoms = mols[molid].GetNumAtoms()
conf = Chem.Conformer(n_atoms)
for i in range(n_atoms):
    conf.SetAtomPosition(i, coords_new[i,:])

_ = mols[molid].AddConformer(conf, assignId=True)

We can now verify how the fragment was aligned to the original molecule:

In [24]:
p = py3Dmol.view()

for i, mol in enumerate(mols[:2]):
    confid = 0
    if i == molid:
        confid = 1
    
    p.addModel(Chem.MolToMolBlock(mol, confId=confid),'sdf')

p.setStyle({"model": 0}, {'stick':{'colorscheme':'lightgreyCarbon'}})
p.setStyle({"model": 1}, {'stick':{'colorscheme':'redCarbon'}})

p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Unfortunately the point cloud alignment is non-deterministic (it is impossible to set a random seed, see [open3d/#288](https://github.com/intel-isl/Open3D/issues/288) and [open3d/#1263](https://github.com/intel-isl/Open3D/issues/1263)) and therefore sometimes the alignment is extremely good while other times it is not.